最終更新日：2023/1/31
# 多クラス分類における評価指標

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import (
    log_loss,
    f1_score,
    confusion_matrix,
    cohen_kappa_score
)

## multi-class accuracy
二値分類のaccuracyを多クラスへ拡張したもの  
scikit-learnのmetricモジュールのaccuracy_scoreで計算可能

## malti-class logloss
loglossをマルチクラスに拡張したもの
$$
    \textrm{multiclass logloss} = -\frac{1}{N} \sum_{i=1}^N \sum_{m=1}^M y_{i,m} \log p_{i,m}
$$
- $M$: クラス数
- $y_{i,m}$: コレード$i$がクラス$m$に属する場合は$1$、それ以外は$0$
- $p_{i,m}$: レコード$i$がクラス$m$に属する予測確率（スコア）  
scikit-learnのmetricのlog_lossで計算可能

In [3]:
# ３クラスの場合
y_true = np.array([0, 2, 1, 2, 2])
y_pred = np.array(      # 予測スコアはレコード数xクラス数の行列
    [
    [0.68, 0.32, 0.00],
    [0.00, 0.00, 1.00],
    [0.60, 0.40, 0.00],
    [0.00, 0.00, 1.00],
    [0.28, 0.12, 0.60]
    ]
)

logloss = log_loss(y_true, y_pred)
print(logloss)

0.3625557672904264


# mean-F1, macro-F1, micro-F1
F1-scoreを多クラス分類に拡張したもの  
マルチラベル分類では各レコードが複数のクラスに属するので、真値と予測値が複数になりうる。

In [10]:
# マルチラベル分類
# マルチクラス分類では1-hotだが、マルチラベルでは各レコードに複数の1が入りうる
# 真値
y_true = np.array(
    [
    [1, 1, 0],
    [1, 0, 0],
    [1, 1, 1],
    [0, 1, 1],
    [0, 0, 1]
    ]
)
# 予測値
y_pred = np.array(
    [
    [1, 0, 1],
    [0, 1, 0],
    [1, 0, 1],
    [0, 0, 1],
    [0, 0, 1]
    ]
) 

# mean-f1は、レコードごとにF1-scoreを計算して平均をとる
mean_f1 = np.mean([f1_score(y_true[i, :], y_pred[i, :]) for i in range(len(y_true))])

# macro_f1では、クラスごとにF1-scoreを計算して平均をとる
n_class = 3
macro_f1 = np.mean([f1_score(y_true[:,c], y_pred[:, c]) for c in range(n_class)])

# micro-f1ではレコードxクラスのペア事にTP/TN/FP/FNを計算しF1-scoreを求める
micro_f1 = f1_score(y_true.reshape(-1), y_pred.reshape(-1))

print(f'mean_f1: {mean_f1:.4f}, macro_f1: {macro_f1:.4f}, micro_f1: {micro_f1:.4f}')


# sckit-leanのメソッドも使える
mean_f1 = f1_score(y_true, y_pred, average='samples')
macro_f1 = f1_score(y_true, y_pred, average='macro')
micro_f1 = f1_score(y_true, y_pred, average='micro')

print(f'mean_f1: {mean_f1:.4f}, macro_f1: {macro_f1:.4f}, micro_f1: {micro_f1:.4f}')

mean_f1: 0.5933, macro_f1: 0.5524, micro_f1: 0.6250
mean_f1: 0.5933, macro_f1: 0.5524, micro_f1: 0.6250


## quadratic weighted kappa
- クラス間に順序関連がある場合で使用
$$
    \kappa = 1 - \frac{\sum_{i,j}w_{i,j}O_{i,j}}{\sum_{i,j}w_{i,j}E_{i,j}}
$$
- $O_{i,j}$: 真のクラスが$i$、予測クラスが$j$のレコード数
- $E_{i,j}$: 真の値のクラスと予測値のクラスの分布が互いに独立であるとした場合に、混合行列の各セル$(i,j)$に属するレコード数
    - 真の値が$i$である割合$\times$予測値が$j$である割合$\times$データ全体のレコード数
- $w_{i,j}$: 真の値と予測値の佐野二乗。$(i-j)^2$
- 完全の予測で$1$、ランダムな予測で$0$、ランダムより悪い予測で負

In [6]:
def quadratic_weighted_kappa(c_matrix):
    numer = 0.
    denom = 0.

    for i in range(c_matrix.shape[0]):
        for j in range(c_matrix.shape[1]):
            n = c_matrix.shape[0]
            wij = ((i - j) ** 2.0)
            oij = c_matrix[i, j]
            eij = c_matrix[i,:].sum() * c_matrix[:,j].sum() / c_matrix.sum()
            numer += wij * oij
            denom += wij * eij

    return 1.0 - numer / denom

y_true = [1, 2, 3, 4, 3]
y_pred = [2, 2, 4, 4, 5]

c_matrix = confusion_matrix(y_true=y_true, y_pred=y_pred, labels=[1, 2, 3, 4, 5])

kappa = quadratic_weighted_kappa(c_matrix)
print(f'kappa: {kappa:.4f}')

# scikit-learn
kappa = cohen_kappa_score(y_true, y_pred, weights='quadratic')
print(f'kappa: {kappa:.4f}')

kappa: 0.6154
kappa: 0.6154


## MAP@K
- レコメンデーションのタスクでよく使われる指標
- 各レコードが１つまたは複数のクラスに属しているときに、属している可能性が高いと予測する順に$K$個のクラスを予測する
$$
    \textrm{MAP@K} = \frac{1}{N} \sum_{i=1}^N \left( \frac{1}{min(m_i, K)} \sum_{k=1}^K P_i(k) \right)
$$
- $m_i$: レコード$i$の属しているクラス数
- $P_i(k)$: レコード$i$について、$k (1 \leq k \leq K)$までの予測値で計算されるprecision
- 正解の予測値が後ろになるとスコアが下がる

In [10]:
# K=3, レコード数は5, クラス数は4
K = 3

# 各レコードの真の値
y_true = [[1, 2], [1, 2], [4], [1, 2, 3, 4], [3, 4]]
y_pred = [[1, 2, 4], [4, 1, 2], [1, 4, 3], [1, 2, 3], [1, 2, 4]]

def apk(y_i_true, y_i_pred):
    assert (len(y_i_pred) <= K)
    assert (len(np.unique(y_i_pred)) == len(y_i_pred))

    sum_precision = 0.0
    num_hits = 0.0

    for i, p in enumerate(y_i_pred):
        if p in y_i_true:
            num_hits += 1
            precision = num_hits / (i + 1)
            sum_precision += precision
    return sum_precision / min(len(y_i_true), K)

def mapk(y_true, y_pred):
    return np.mean([apk(y_i_true, y_i_pred) for y_i_true, y_i_pred in zip(y_true, y_pred)])

print(f'MAP@k: {mapk(y_true, y_pred):.4f}')

# 正解の数が同じでも順位が異なると違う値になる
print(f'AP@k: {apk(y_true[0], y_pred[0]):.4f}')
print(f'AP@k: {apk(y_true[1], y_pred[1]):.4f}')

MAP@k: 0.6500
AP@k: 1.0000
AP@k: 0.5833
